```
This script can be used for any purpose without limitation subject to the
conditions at http://www.ccdc.cam.ac.uk/Community/Pages/Licences/v2.aspx

This permission notice and the following statement of attribution must be
included in all copies or substantial portions of this script.

2024-09-11: Made available by the Cambridge Crystallographic Data Centre.

```

# International Report

This notebook talks through how to create a report that reports a CSD entry ... but uses google translate to translate headings into the language of user choice.
You could use the ideas her to convert the 'simple_report.py script into an international report

#### Prerequisites

First install the CSD Python API and googletrans into your conda or pip environment. This should only be needed the before you run the script.

Now we are ready to start some python coding. We need to import some modules. We are writing a script that ultimately will be used in Mercury so we will want some of 
the special utilities available for writing reports.

In [ ]:
from ccdc.utilities import html_table


We can create a translator for the script to use, and define our language of choice. Lets write a little function to do the translation

In [ ]:
from googletrans import Translator

translator = Translator()
def tr(text, lang):
    if lang is None:
        return text
    try:
        return translator.translate(str(text),src="en",dest=lang).text
    except Exception as e:
        return text

Let's try it out:

In [ ]:

tr("Mary had a little lamb", "ar")  # Lets try arabic!

So now we can create a report on a CSD entry that is international. The following function will work in a Mercury API script. Note how it defines an interface object

In [ ]:
def get_coordinates(molecule, round_digits=None):
    """Yield the label and fractional coordinates of all atoms in the molecule.

    :param molecule: (:obj:`ccdc.molecule.Molecule`) The molecule for which to return coordinates.
    :param round_digits: (:obj:`int`) How many decimal digits coordinates should be rounded to.
    :returns: (:obj:`list`) List of the label and fractional x/y/z coordinates for each atom
        in the molecule in format ['Atom label', 'X coordinate', 'Y coordinate', 'Z coordinate'].
    """
    for atom in molecule.atoms:
        try:
            x, y, z = atom.fractional_coordinates
            yield [atom.label,
                   x if round_digits is None else round(x, round_digits),
                   y if round_digits is None else round(y, round_digits),
                   z if round_digits is None else round(z, round_digits)]
        except TypeError:
            continue


def main(interface=None, lang=None):
    """Generate a simple report based on the entry currently selected in the Mercury UI.

    :param interface: (:obj:`ccdc.utility.ApplicationInterface`) An ApplicationInterface instance.
    """
    if interface is None:
        from ccdc.utilities import ApplicationInterface
        interface = ApplicationInterface()

    entry = interface.current_entry

    # Open a HTML report. This will create the file, copy the CSD Python API
    # default template for reports and fill in the headline/page title.
    with interface.html_report(title=tr('Simple report on ',lang)+ ' ' + entry.identifier) as report:

        # Write the section header for Entry Details
        report.write_section_header(tr('Entry Details',lang))
        # Assemble a list of information and labels to go into the "Entry Details" table
        entry_details = [
            ['<b>'+tr('Chemical name',lang)+'</b>', entry.chemical_name],
            ['<b>'+tr('Synonyms',lang)+'</b>', entry.synonyms],
            ['<b>'+tr('Formula',lang)+'</b>', entry.formula],
            ['<b>'+tr('R-factor',lang)+'</b>', entry.r_factor],
            ['<b>'+tr('Disorder',lang)+'</b>', tr(entry.disorder_details,lang)],
            ['<b>'+tr('Polymorphism',lang)+'</b>', tr(entry.polymorph,lang)],
            ['<b>'+tr('3D structure',lang)+'</b>', tr(entry.has_3d_structure,lang)],
            ['<b>'+tr('Organic',lang)+'</b>', tr(entry.is_organic,lang)],
            ['<b>'+tr('Polymeric',lang)+'</b>', tr(entry.is_polymeric,lang)],
            ['<b>'+tr('Bioactivity',lang)+'</b>', tr(entry.bioactivity,lang)],
            ['<b>'+tr('Source',lang)+'</b>', tr(entry.source,lang)],
            ['<b>'+tr('Habit',lang)+'</b>', tr(entry.habit,lang)],
        ]
        # Generate a HTML table from the entry details and write it to the report
        report.write(html_table(data=entry_details, table_id='entry_details'))

        # Write the section header for Fractional Coordinates
        report.write_section_header(tr('Fractional Coordinates',lang))
        # Get the coordinates of all the atoms from the entry and write them to a HTML table
        report.write(html_table(data=list(get_coordinates(entry.molecule, round_digits=3)),
                                table_id='fractional_coordinates',
                                header=[tr('Atom',lang), 'x', 'y', 'z']))

        # Write the section header for Publication Details
        report.write_section_header(tr('Publication Details',lang))
        # Assemble a list of information and labels for the Publication Details table
        publication_details = [
            ['<b>'+tr('Reference',lang)+'</b>', '%s Volume %s, %s' % (getattr(entry.publication, 'journal_name', ''),
                                                       entry.publication.volume,
                                                       entry.publication.year)],
            ['<b>'+tr('Authors', lang)+'</b>', entry.publication.authors],
            ['<b>'+tr('Document Object Identifier',lang)+'</b>', entry.publication.doi]
        ]
        # Write the publication details to a HTML table
        report.write(html_table(publication_details, table_id='publication_details'))

        # Write the section header for Basic Crystallographic Information
        report.write_section_header(tr('Basic Crystallographic Information',lang))
        # Assemble a list of basic crystal information and labels for the table
        crystallographic_data = [
            ['<b>'+tr('Crystal System',lang)+'</b>', entry.crystal.crystal_system],
            ['<b>'+tr('Space Group',lang) + '</b>', entry.crystal.spacegroup_symbol],
            ['<b>'+tr('Cell Volume',lang)+ '</b>', '%s Å³' % round(entry.crystal.cell_volume, 3)],
            ['<b>Z, Z\'</b>', entry.crystal.z_prime],
        ]
        # Write the crystallographic details to a HTML table
        report.write(html_table(crystallographic_data, table_id='crystallographic_information'))

    # Once the HTMLReport is closed (e.g. when the with: branch above ends),
    # it will automatically write the appropriate HTML footer.

Lets use our function. As we are in a notebook, we will have to define a dummy interface file:

In [ ]:
import time
from ccdc.utilities import ApplicationInterface
interface = ApplicationInterface(parse_commandline=False)
interface.identifier = "AABHTZ"
interface.output_html_file = f'{interface.identifier}_{time.strftime("%H%M%S", time.gmtime())}_report.html'

main(interface,"ar") # Arabic 

Finally use Jupyter to display it:

In [ ]:
from IPython.display import HTML
HTML(interface.output_html_file)